In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification

from scipy.stats import loguniform

from tpot import TPOTClassifier, TPOTRegressor
from deap.gp import Primitive

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


---

1. С помощью make_classification из sklearn.datasets сгенерируйте датасет с 1 000 объектов, 10 признаками и бинарным таргетом.

In [2]:
# Генерация датасета
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    #n_classes=2,
    #random_state=42,
)

print(X.shape, y.shape)

(1000, 10) (1000,)


---

2. Зададим сетку гиперпараметров и инициализируем модель в виде пайплайна. Параметр penalty будем выбирать равновероятно из  ['l1', 'l2'], а параметр регуляризации С из лог-равномерного распределения может принимать значения [10-4, 102].

In [3]:
search_space = {
                'lr__penalty' : ['l1', 'l2'],
                'lr__C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

model = Pipeline([('lr', LogisticRegression(random_state=42, 
                            solver='liblinear'))])

---

Перебор по сетке будем выполнять с помощью GridSearchCV из sklearn.model_selection. 

3. При его инициализации укажем несколько параметров: 

Модель (пайплайн)
Сетку с параметрами
Количество разбиений для (Stratified)KFold, который используется по умолчанию = 3
Метрику для оценки производительности модели с перекрёстной проверкой на тестовом наборе scoring='accuracy'.

4. Обучите GridSearchCV с параметрами (model_grid = grid_search.fit(X, y)).

In [4]:
model_grid = GridSearchCV(
    estimator=model,
    param_grid=search_space,
    cv=3,
    scoring='accuracy'
)

model_grid.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('lr',
                                        LogisticRegression(random_state=42,
                                                           solver='liblinear'))]),
             param_grid={'lr__C': array([3.63735966e-04, 8.47029461e-02, 9.50018429e-03, 1.83184779e-01,
       2.62618269e-03, 1.21323340e+01, 1.50719994e-04, 1.40496547e-01,
       1.45366475e+00, 1.30240716e-01, 4.60995298e+01, 2.54841297e+00,
       4.67914774e-02, 1.35644883e-03, 1.051...
       4.76067252e-01, 5.84027233e+01, 1.76609306e+00, 7.74536475e-02,
       1.83375051e-04, 1.44423825e-04, 1.06025555e-01, 7.44286622e+00,
       5.17298060e-04, 2.07567944e-04, 4.31203045e+01, 1.18039692e+00,
       8.55867564e-03, 1.45137106e+01, 9.06189884e-03, 7.74139883e-01,
       1.14450314e-04, 1.93567465e+00, 4.38672499e+00, 4.07234833e-04,
       8.74726578e+01, 2.26034383e+01, 1.79240372e+01, 7.92416461e-03]),
                         'lr__penalty': ['l1', 'l2']},
             scoring='accuracy')

Теперь можем посмотреть на лучший score и наилучшие гиперпараметры:

In [5]:
print(model_grid.best_score_)
print(model_grid.best_params_)

0.8649787512062961
{'lr__C': 0.0057650777331017975, 'lr__penalty': 'l2'}


---

Для случайного поиска воспользуемся RandomizedSearchCV из sklearn.model_selection. Помимо модели, параметров, скоринга и cv, зададим n_iter. Он отвечает за количество выбранных комбинаций параметров. Чем больше n_iter, тем дольше будет работать поиск. Соответственно, максимально возможный n_iter приближает RandomizedSearchCV к GridSearchCV. 

6. Задайте n_iter = 70 и инициализируйте RandomizedSearchCV.

7. Обучите полученный оптимизатор на X и y и оцените .best_score_ и .best_params_.

In [6]:
model_random = RandomizedSearchCV(
    estimator=model,
    param_distributions=search_space,
    cv=3,
    scoring='accuracy',
    n_iter=70
)

model_random.fit(X, y)

print(model_random.best_score_)
print(model_random.best_params_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.8639837442232651
{'lr__penalty': 'l1', 'lr__C': 0.04471384829815968}


---

### TPOT (Работает под 3.9)

Для генетического алгоритма воспользуемся библиотекой TPOT.

8. Для установки выполните pip install tpot, после чего импортируйте из tpot TPOTClassifier.

9. Задайте параметры немного иначе, при этом оставив те же значения:

In [3]:
search_space = {
                'penalty' : ['l1', 'l2'],
                'C' : loguniform.rvs(10**(-4),10**2, size=100)
                }



10. При инициализации классификатора добавьте следующие параметры:

generations = 5, (Количество поколений в процессе оптимизации)

population_size = 50, (Число особей, сохраняемых в популяции генетического программирования в каждом поколении)

offspring_size = 25, (Количество потомства, которое нужно произвести в каждом поколении генетического программирования)

verbosity = 2, 

config_dict = {'sklearn.linear_model.LogisticRegression': search_space}, (словарь с гиперпараметрами для оптимизации для выбранной модели)

cv = 3, 

scoring = 'accuracy'

In [1]:
# Создание экземпляра TPOTClassifier с заданными параметрами
tpot_classifier = TPOTClassifier(
    generations=5,
    population_size=50,
    offspring_size=25,
    early_stop=8,
    verbosity=2,
    config_dict = {'sklearn.linear_model.LogisticRegression': search_space},
    cv=3,
    scoring="accuracy",
)

NameError: name 'TPOTClassifier' is not defined

11. Обучите инициализированный классификатор tpot_classifier.fit(X, y).

In [5]:
tpot_classifier.fit(X,y)

Optimization Progress:   0%|          | 0/175 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -inf


RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly. If you enabled PyTorch estimators, please check the data requirements in the online documentation: https://epistasislab.github.io/tpot/using/

In [10]:
args = {}
for arg in tpot_classifier._optimized_pipeline:
    if type(arg) != Primitive:
        try:
            if arg.value.split('__')[1].split('=')[0] in ['C', 'penalty']:
                args[arg.value.split('__')[1].split('=')[0]] = (arg.value.split('__')[1].split('=')[1])
            else:
                args[arg.value.split('__')[1].split('=')[0]] = float(arg.value.split('__')[1].split('=')[1])
        except:
            pass
params = args

params

TypeError: 'NoneType' object is not iterable

---

### БАЙЕСОВСКАЯ ОПТИМИЗАЦИЯ (Работает под 3.11)

Мы рассмотрим библиотеку Hyperopt для подбора гиперпарметров. В ней реализовано три алгоритма оптимизации: 

классический Random Search;

метод байесовской оптимизации Tree of Parzen Estimators (TPE);

Simulated Annealing, метод имитации отжига. 


Hyperopt может работать с разными типами гиперпараметров — непрерывными, дискретными, категориальными и так далее, что является важным преимуществом этой библиотеки.

14. Импортируйте вспомогательные функции:

In [2]:
from functools import partial
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

15. Укажите объект для сохранения истории поиска (Trials). Это очень удобно, поскольку можно сохранять, прерывать и затем продолжать процесс поиска гиперпараметров. Запустите сам процесс подбора с помощью функции fmin. Укажите в качестве алгоритма поиска tpe.suggest — байесовскую оптимизацию. Для Random Search нужно указать tpe.rand.suggest.

In [7]:
# Генерация датасета
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    #n_classes=2,
    #random_state=42,
)

print(X.shape, y.shape)


trials = Trials()

model = Pipeline([('lr', LogisticRegression(random_state=42, 
                            solver='liblinear'))])

(1000, 10) (1000,)


16. Задайте гиперпараметры с использованием распределения из hyperopt:

In [8]:
search_space = {
                'lr__penalty' : hp.choice(label='penalty', 
                          options=['l1', 'l2']),
                'lr__C' : hp.loguniform(label='C', 
                        low=-4*np.log(10), 
                        high=2*np.log(10))}

17. Нам понадобится воспользоваться вспомогательной функцией, которую мы будем оптимизировать:

In [9]:
def objective(params, model,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """ 

    # задаём модели требуемые параметры    
    model.set_params(**params)
    
    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

    # проводим кросс-валидацию  
    score = cross_val_score(estimator=model, X=X_train, y=y_train, 
                            scoring='accuracy', cv=skf, n_jobs=-1)

    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

18. Запустите hyperopt.

In [10]:
best = fmin( 
          # функция для оптимизации  
            fn=partial(objective, model=model, X_train=X, y_train=y),
          # пространство поиска гиперпараметров  
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций (можно ещё указать  время поиска) 
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            #rstate=np.random.RandomState(42),
          # progressbar
            show_progressbar=True
        )

  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 40/40 [00:01<00:00, 20.00trial/s, best loss: -0.894]


19. Посмотрите на параметры, подобранные с помощью байесовской оптимизации, и сравните с предыдущими результатами. Какая модель имеет наилучший score на кросс-валидации?

In [11]:
print(best)

{'C': 0.044834342948026636, 'penalty': 1}
